Dependences to install

In [13]:
%pip install tensorflow==2.3.1
%pip install gym
%pip install keras-rl2
%pip install gym[atari]

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.9.0rc0)
ERROR: No matching distribution found for tensorflow==2.3.1


  Using cached gym-0.23.1-py3-none-any.whl
  Using cached gym_notices-0.0.6-py3-none-any.whl (2.7 kB)
Note: you may need to restart the kernel to use updated packages.

  Using cached ale_py-0.7.4-cp39-cp39-win_amd64.whl (904 kB)



Environment : OpenAI Gym

In [1]:
import random
import gym
gym.__file__


'C:\\Users\\AbdelfataH\\anaconda3\\envs\\gym\\lib\\site-packages\\gym\\__init__.py'

In [2]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    scoure = 0
    while not done:
        env.render()
        action = random.choice([0, 1, 2, 3, 4, 5])
        n_state, reward, done, info = env.step(action)
        scoure += reward
    print('Episode: {} Scoure: {}'.format(episode, scoure))
env.close()

Episode: 1 Scoure: 120.0
Episode: 2 Scoure: 65.0
Episode: 3 Scoure: 135.0
Episode: 4 Scoure: 330.0
Episode: 5 Scoure: 110.0


Deep Learning Model : Keras

In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [7]:
model = build_model(height, width, channels, actions)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

In [ ]:
del model

Build Agent : Keras-RL

In [9]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [11]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit = 1000, window_length = 3)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, 
                    enable_dueling_network=True, dueling_type='avg',
                    nb_actions = actions, nb_steps_warmup = 1000)
    return dqn

In [13]:
model = build_model(height, width, channels, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


C:\Users\AbdelfataH\anaconda3\envs\gym\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  605/10000: episode: 1, duration: 50.927s, episode steps: 605, steps per second:  12, episode reward: 120.000, mean reward:  0.198 [ 0.000, 30.000], mean action: 2.374 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


C:\Users\AbdelfataH\anaconda3\envs\gym\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1671/10000: episode: 2, duration: 2127.889s, episode steps: 1066, steps per second:   1, episode reward: 410.000, mean reward:  0.385 [ 0.000, 200.000], mean action: 2.498 [0.000, 5.000],  loss: 11.893722, mean_q: 3.931121, mean_eps: 0.879805
 2269/10000: episode: 3, duration: 1780.524s, episode steps: 598, steps per second:   0, episode reward: 120.000, mean reward:  0.201 [ 0.000, 30.000], mean action: 2.411 [0.000, 5.000],  loss: 1.314655, mean_q: 3.874373, mean_eps: 0.822745
 2898/10000: episode: 4, duration: 1923.912s, episode steps: 629, steps per second:   0, episode reward: 90.000, mean reward:  0.143 [ 0.000, 30.000], mean action: 2.603 [0.000, 5.000],  loss: 0.865693, mean_q: 2.314277, mean_eps: 0.767530
 3310/10000: episode: 5, duration: 1266.908s, episode steps: 412, steps per second:   0, episode reward: 45.000, mean reward:  0.109 [ 0.000, 15.000], mean action: 2.551 [0.000, 5.000],  loss: 0.586012, mean_q: 2.451395, mean_eps: 0.720685
 3915/10000: episode: 6, duration: 

AbortedError: Operation received an exception:Status: 1, message: could not execute a primitive, in file tensorflow/core/kernels/mkl/mkl_matmul_op_fused.cc:260
	 [[{{node dense_7_1/Relu}}]]

Reloading Agent from Memory